<a href="https://colab.research.google.com/github/Gowrisan1987/NLP/blob/master/Youtube_comments_spam_collection_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
#import spacy
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import glob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
path='/content/drive/My Drive/Data/'

In [4]:
# Source: Downloaded from UCI Machine learning repository
yt_df = pd.concat([pd.read_csv(f,header=0) for f in glob.glob(path+"/Youtube*.csv")], ignore_index = True)

In [ ]:
#yt_df=pd.read_csv("/content/drive/My Drive/Data/Youtube04-Eminem.csv")

In [10]:
yt_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
yt_df.isnull().values.any()

True

In [7]:
yt_df['CONTENT'].isnull().values.any()

False

In [8]:
yt_df['CLASS'].value_counts()

1    1005
0     951
Name: CLASS, dtype: int64

In [11]:
yt_df.shape

(1956, 5)

In [ ]:
#nlp=spacy.load('en_core_web_sm')

In [ ]:
#x=yt_df.drop('CLASS',axis=1)

In [12]:
y=yt_df['CLASS']

In [13]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(yt_df)):
    review = re.sub('[^a-zA-Z]', ' ', yt_df['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
#def clean_text(token):
  #if token.is_stop==False and token.is_punct==False and not token.is_space and token.lemma_!='-PRON-':
  #  return True
  #return False

#def lemma_text(text):
    #cleaned_text=[token.lemma_.lower() for token in nlp(text) if clean_text(token)]
    #return cleaned_text


# Cleaning through Spacy utilities
#def cleaned_text(data):
  #for i in range(0,1):
   # text = []
    #doc=nlp(data)
    #for j in range(0,len(doc)):
      #print(j)
    #token=[word.lemma_.lower().strip() for word in doc if word.lemma_ != '-PRON-' and word.is_stop==False and word.is_punct==False]
    #token=[word for word in token if word.is_stop==False and token.is_punct==False]
    #token = ' '.join(token)
    #text.append(token)

  #return text
  
#test1=yt_df['CONTENT'][2]
#test2=cleaned_text(test1)


In [ ]:
#text = []

#for i in range(0,len(yt_df)):
  #doc=nlp(yt_df['CONTENT'][i])
  #token=[word.lemma_.lower().strip() for word in doc if word.is_stop==False and word.is_punct==False and word.is_alpha==True]
  #token = ' '.join(token)
  
  #text.append(token)


In [14]:
voc_size=5000

In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [16]:
onehot_repr[0]

[2679, 448, 2222, 838, 1518, 1437]

In [19]:
## Padding to keep corpus of same length as embedded layer requires input of same length
sent_ln=20
padded_corpus=pad_sequences(onehot_repr,maxlen=sent_ln,padding='pre')

In [20]:
padded_corpus[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 2679,  448, 2222,  838, 1518, 1437], dtype=int32)

# Using LSTM approach

In [21]:
model=Sequential()
model.add(Embedding(voc_size,50,input_length=sent_ln))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            250000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
x_final=np.array(padded_corpus)
y_final=np.array(y)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.20, random_state=42)

In [24]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
25/25 [==============================] - 1s 55ms/step - loss: 0.6449 - accuracy: 0.6451 - val_loss: 0.5578 - val_accuracy: 0.6990
Epoch 2/10
25/25 [==============================] - 1s 34ms/step - loss: 0.4403 - accuracy: 0.7826 - val_loss: 0.3106 - val_accuracy: 0.9082
Epoch 3/10
25/25 [==============================] - 1s 35ms/step - loss: 0.2113 - accuracy: 0.9239 - val_loss: 0.2015 - val_accuracy: 0.9439
Epoch 4/10
25/25 [==============================] - 1s 36ms/step - loss: 0.1183 - accuracy: 0.9642 - val_loss: 0.1683 - val_accuracy: 0.9541
Epoch 5/10
25/25 [==============================] - 1s 34ms/step - loss: 0.0736 - accuracy: 0.9776 - val_loss: 0.1645 - val_accuracy: 0.9592
Epoch 6/10
25/25 [==============================] - 1s 34ms/step - loss: 0.0480 - accuracy: 0.9847 - val_loss: 0.1631 - val_accuracy: 0.9566
Epoch 7/10
25/25 [==============================] - 1s 33ms/step - loss: 0.0313 - accuracy: 0.9885 - val_loss: 0.1831 - val_accuracy: 0.9566
Epoch 8/10
25

In [25]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [26]:
confusion_matrix(y_test,y_pred)

array([[161,  15],
       [  7, 209]])

In [27]:
accuracy_score(y_test,y_pred)

0.9438775510204082

## Trying using Bidirectional LSTM

In [29]:
model1=Sequential()
model1.add(Embedding(voc_size,50,input_length=sent_ln))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
25/25 [==============================] - 2s 91ms/step - loss: 0.6082 - accuracy: 0.6861 - val_loss: 0.3680 - val_accuracy: 0.8367
Epoch 2/10
25/25 [==============================] - 1s 54ms/step - loss: 0.2290 - accuracy: 0.9111 - val_loss: 0.1753 - val_accuracy: 0.9439
Epoch 3/10
25/25 [==============================] - 1s 48ms/step - loss: 0.0847 - accuracy: 0.9763 - val_loss: 0.1686 - val_accuracy: 0.9439
Epoch 4/10
25/25 [==============================] - 1s 49ms/step - loss: 0.0355 - accuracy: 0.9917 - val_loss: 0.2464 - val_accuracy: 0.9362
Epoch 5/10
25/25 [==============================] - 1s 49ms/step - loss: 0.0370 - accuracy: 0.9859 - val_loss: 0.2812 - val_accuracy: 0.9235
Epoch 6/10
25/25 [==============================] - 1s 49ms/step - loss: 0.0208 - accuracy: 0.9936 - val_loss: 0.2093 - val_accuracy: 0.9490
Epoch 7/10
25/25 [==============================] - 1s 50ms/step - loss: 0.0147 - accuracy: 0.9962 - val_loss: 0.2555 - val_accuracy: 0.9464
Epoch 8/10
25

In [31]:
y_pred_model1=model1.predict_classes(X_test)

In [32]:
confusion_matrix(y_test,y_pred_model1)

array([[165,  11],
       [ 15, 201]])

In [33]:
accuracy_score(y_test,y_pred_model1)

0.9336734693877551